## Import libraries

In [57]:
import warnings
warnings.filterwarnings("ignore")

In [58]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

## Anomaly-based IDS - Infiltration

### Generate the port-scan datasets for unknown attack detection

In [222]:
df=pd.read_csv('CICIDS2018_sample_km.csv')

In [223]:
df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.475925,-0.005346,-0.020438,-0.040784,-0.020339,-0.022177,-0.830722,-0.361301,-0.961988,-0.776052,...,0.187746,-0.070847,-0.057248,-0.085188,-0.058141,0.007993,-0.001875,0.002505,0.039759,6
1,-0.475925,-0.005361,-0.020438,-0.040784,-0.020339,-0.022177,-0.830722,-0.361301,-0.961988,-0.776052,...,0.187746,-0.070847,-0.057248,-0.085188,-0.058141,0.007993,-0.001875,0.002505,0.039759,6
2,-0.475925,-0.005399,-0.020438,-0.040784,-0.020339,-0.022177,-0.830722,-0.361301,-0.961988,-0.776052,...,0.187746,-0.070847,-0.057248,-0.085188,-0.058141,0.007750,-0.001875,0.002402,0.038739,6
3,-0.475925,-0.005334,-0.020438,-0.040784,-0.020339,-0.022177,-0.830722,-0.361301,-0.961988,-0.776052,...,0.187746,-0.070847,-0.057248,-0.085188,-0.058141,0.007993,-0.001875,0.002505,0.039759,6
4,-0.475925,-0.005453,-0.020438,-0.040784,-0.020339,-0.022177,-0.830722,-0.361301,-0.961988,-0.776052,...,0.187746,-0.070847,-0.057248,-0.085188,-0.058141,0.007750,-0.001875,0.002402,0.038739,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1.944820,-0.018959,-0.020859,-0.035722,-0.019875,-0.021916,-0.704356,1.437948,-0.330935,-0.776052,...,-1.492618,-0.070847,-0.057248,-0.085188,-0.058141,-0.017585,-0.001875,-0.008251,-0.067285,0
36453,1.944820,-0.018959,-0.020859,-0.035722,-0.019875,-0.021916,-0.704356,1.437948,-0.330935,-0.776052,...,-1.492618,-0.070847,-0.057248,-0.085188,-0.058141,-0.017585,-0.001875,-0.008251,-0.067285,0
36454,1.944820,-0.018959,-0.020859,-0.035722,-0.019875,-0.021916,-0.704356,1.437948,-0.330935,-0.776052,...,-1.492618,-0.070847,-0.057248,-0.085188,-0.058141,-0.017585,-0.001875,-0.008251,-0.067285,0
36455,1.944820,-0.018959,-0.020859,-0.035722,-0.019885,-0.021920,-0.707164,1.397965,-0.344959,-0.776052,...,-1.492618,-0.070847,-0.057248,-0.085188,-0.058141,-0.017585,-0.001875,-0.008251,-0.067285,0


In [224]:
df.Label.value_counts()

,count
Label,
0,10663
10,9908
6,5748
4,1989
5,1730
8,1448
1,1440
12,1183
14,943


In [225]:
df1 = df[df['Label'] != 2]
df1['Label'][df1['Label'] > 0] = 1
df1.to_csv('./CICIDS2017_sample_km_without_infiltration.csv',index=0)

In [226]:
df2 = df[df['Label'] == 2]
df2['Label'][df2['Label'] == 2] = 1
df2.to_csv('./CICIDS2017_sample_km_infiltration.csv',index=0)

### Read the generated datasets for unknown attack detection

In [227]:
df1 = pd.read_csv('./CICIDS2017_sample_km_without_infiltration.csv')
df2 = pd.read_csv('./CICIDS2017_sample_km_infiltration.csv')

In [228]:
features = df1.drop(['Label'],axis=1).dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df2[features] = df2[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [229]:
df1.Label.value_counts()

,count
Label,
1,25226
0,10663


In [230]:
df2.Label.value_counts()

,count
Label,
1,568


In [231]:
df2p=df1[df1['Label']==0]
df2pp=df2p.sample(n=None, frac=568/10663, replace=False, weights=None, random_state=None, axis=0)
df2=pd.concat([df2, df2pp])

In [232]:
df2.Label.value_counts()

,count
Label,
1,568
0,568


In [233]:
df = pd.concat([df1, df2])

In [234]:
df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.380542,-0.514608,-0.219098,-0.069442,-0.219880,-0.028054,-0.727419,-0.300677,-0.903180,-0.715422,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.127906,-0.390973,-0.238288,-0.047757,1
1,-0.380542,-0.514849,-0.219098,-0.069442,-0.219880,-0.028054,-0.727419,-0.300677,-0.903180,-0.715422,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.127906,-0.390973,-0.238288,-0.047757,1
2,-0.380542,-0.515467,-0.219098,-0.069442,-0.219880,-0.028054,-0.727419,-0.300677,-0.903180,-0.715422,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.131450,-0.390973,-0.241506,-0.051325,1
3,-0.380542,-0.514413,-0.219098,-0.069442,-0.219880,-0.028054,-0.727419,-0.300677,-0.903180,-0.715422,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.127906,-0.390973,-0.238288,-0.047757,1
4,-0.380542,-0.516338,-0.219098,-0.069442,-0.219880,-0.028054,-0.727419,-0.300677,-0.903180,-0.715422,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.131450,-0.390973,-0.241506,-0.051325,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25571,-0.380542,-0.609030,-0.219021,-0.011629,-0.219763,-0.025741,-0.361699,-0.300677,-0.561061,-0.266899,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.500035,-0.390973,-0.576188,-0.422358,0
29885,-0.380542,1.578679,-0.218443,0.219621,-0.219467,0.006328,0.053035,-0.300677,-0.648494,-0.215774,...,-0.374398,-0.322078,-0.209249,-0.244827,-0.327634,-0.148410,-0.352208,-0.254038,-0.097183,0
33470,2.515906,-0.734378,-0.219137,-0.054989,-0.219821,-0.026953,-0.542674,1.954782,-0.211889,-0.715422,...,-1.695918,-0.360122,-0.289961,-0.359587,-0.345991,-0.500035,-0.390973,-0.576188,-0.422358,0
34434,-0.380542,-0.734651,-0.218983,-0.040536,-0.218754,-0.025371,2.797817,-0.300677,1.735014,3.151536,...,-0.374398,-0.360122,-0.289961,-0.359587,-0.345991,-0.500035,-0.390973,-0.576188,-0.422358,0


In [235]:
X = df.drop(['Label'],axis=1) .values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)
pd.Series(y).value_counts()

,count
1,25794
0,11231


### Feature engineering (IG, FCBF, and KPCA)

#### Feature selection by information gain (IG)

In [236]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X, y)

In [237]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [238]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break

In [239]:
X_fs = df[fs].values

In [240]:
X_fs.shape

(37025, 45)

In [241]:
X_fs

array([[-0.21988013, -0.21988013, -0.72741873, ..., -0.06944201,
        -0.42592241, -0.23828778],
       [-0.21988013, -0.21988013, -0.72741873, ..., -0.06944201,
        -0.42592241, -0.23828778],
       [-0.21988013, -0.21988013, -0.72741873, ..., -0.06944201,
        -0.42592241, -0.24150588],
       ...,
       [-0.21982113, -0.21982113, -0.54267373, ..., -0.05498884,
        -0.42592241, -0.57618826],
       [-0.21875439, -0.21875439,  2.79781748, ..., -0.04053566,
        -0.42592241, -0.57618826],
       [-0.21982956, -0.21982956, -0.56906588, ..., -0.05498884,
        -0.42592241, -0.57618826]])

#### Feature selection by Fast Correlation Based Filter (FCBF)

The module is imported from the GitHub repo: https://github.com/SantiagoEG/FCBF_module

In [33]:
!pip install scikit-optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 519.9 kB/s eta 0:00:00


In [242]:
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def fcbf_objective(threshold, X, y, clf=RandomForestClassifier(random_state=42)):
    selector = FCBF(th=threshold)
    X_selected = selector.fit_transform(X, y)
    if X_selected.shape[1] == 0:
        return 1.0
    score = cross_val_score(clf, X_selected, y, cv=3, scoring="accuracy").mean()
    return -score

In [243]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBF(th = 0.09988304703442026)

In [244]:
X_fss = fcbf.fit_transform(X_fs,y)

In [245]:
X_fss.shape

(37025, 1)

In [246]:
X_fss

array([[-0.67902919],
       [-0.67902919],
       [-0.67902919],
       ...,
       [-0.44486526],
       [-0.10852071],
       [-0.38313114]])

####  kernel principal component analysis (KPCA)

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def kpca_objective(params, X, y):
    n_components, kernel = params
    n_components = min(n_components, X.shape[1] - 1)
    if n_components < 1: n_components = 1
    try:
        kpca = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=False, random_state=42)
        X_kpca = kpca.fit_transform(X)
        clf = RandomForestClassifier(random_state=42)
        score = cross_val_score(clf, X_kpca, y, cv=3, scoring="accuracy").mean()
        return -score
    except Exception as e:
        return 1.0

search_space = [
    Integer(10, 50),
    Categorical(['rbf', 'poly'])
]

result = gp_minimize(
    lambda params: kpca_objective(params, X_fss, y),
    search_space,
    n_calls=20,
    random_state=42,
    acq_func='EI'
)

best_n_components, best_kernel = result.x
print(f"Best n_components: {best_n_components}, Best kernel: {best_kernel}")
print(f"Best accuracy: {-result.fun:.4f}")

In [247]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

kpca = KernelPCA(n_components=20, kernel='rbf', random_state=42)
X_kpca = kpca.fit_transform(X_fss)

In [248]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

In [249]:
pd.Series(y_train).value_counts()

,count
1,25226
0,10663


In [250]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:25794})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [251]:
pd.Series(y_train).value_counts()

,count
0,25794
1,25226


In [252]:
pd.Series(y_test).value_counts()

,count
1,568
0,568


### Apply the cluster labeling (CL) k-means method

In [253]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN,MeanShift
from sklearn.cluster import SpectralClustering,AgglomerativeClustering,AffinityPropagation,Birch,MiniBatchKMeans,MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [254]:
def CL_kmeans(X_train, X_test, y_train, y_test,n,b=100):
    km_cluster = MiniBatchKMeans(n_clusters=n,batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_test)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm=confusion_matrix(y_test,result2)
    acc=metrics.accuracy_score(y_test,result2)
    print(str(acc))
    print(cm)

In [255]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:25794})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [256]:
CL_kmeans(X_train, X_test, y_train, y_test, 8)

              precision    recall  f1-score   support

           0       0.76      0.75      0.76       568
           1       0.75      0.76      0.76       568

    accuracy                           0.76      1136
   macro avg       0.76      0.76      0.76      1136
weighted avg       0.76      0.76      0.76      1136

0.7561619718309859
[[428 140]
 [137 431]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [257]:
#Hyperparameter optimization by BO-GP
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space  = [Integer(2, 50, name='n_clusters')]
@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n=params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_val)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_val)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    cm=metrics.accuracy_score(y_val,result2)
    print(str(n)+" "+str(cm))
    return (1-cm)
from skopt import gp_minimize
import time
t1=time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2=time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.9012259682362775
43 0.8931457230426303
43 0.8721092226246866
43 0.8959320144887155
32 0.8959320144887155
20 0.8560880468096963
16 0.8519086096405684
5 0.8329618278071886
15 0.8542769573697409
25 0.8590136528280858
36 0.8931457230426303
50 0.8222346057397604
39 0.8949568124825856
34 0.8916132627472834
39 0.8761493452215102
31 0.8920312064641962
33 0.8581777653942603
38 0.8775424909445528
29 0.8644469211479521
37 0.8619392588464754
7.131067752838135
Best score=0.9012
Best parameters: n_clusters=30


In [258]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:25794})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [259]:
CL_kmeans(X_train, X_test, y_train, y_test, res_gp.x[0])

              precision    recall  f1-score   support

           0       0.75      0.74      0.75       568
           1       0.74      0.76      0.75       568

    accuracy                           0.75      1136
   macro avg       0.75      0.75      0.75      1136
weighted avg       0.75      0.75      0.75      1136

0.7482394366197183
[[419 149]
 [137 431]]


### Apply the CL-k-means model with biased classifiers

95% of the code has been shared, and the remaining 5% is retained for future extension.  
Thank you for your interest and more details are in the paper.

In [260]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer
from imblearn.over_sampling import SMOTE

def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    km = MiniBatchKMeans(n_clusters=n, batch_size=b, random_state=0)
    train_labels = km.fit_predict(X_train)
    test_labels  = km.predict(X_test)

    counts_pos = np.zeros(n)
    counts_neg = np.zeros(n)
    for idx, c in enumerate(train_labels):
        if y_train[idx] == 1: counts_pos[c] += 1
        else:               counts_neg[c] += 1

    cluster_prob = {}
    normal_clusters = []
    attack_clusters = []
    for c in range(n):
        tot = counts_pos[c] + counts_neg[c]
        if counts_pos[c] > counts_neg[c]:
            attack_clusters.append(c)
            cluster_prob[c] = counts_pos[c] / tot if tot>0 else 0.0
        else:
            normal_clusters.append(c)
            cluster_prob[c] = counts_neg[c] / tot if tot>0 else 0.0

    y_km = np.array([1 if c in attack_clusters else 0 for c in test_labels])

    print("CL-k-means Performance:")
    print(classification_report(y_test, y_km))
    cm = confusion_matrix(y_test, y_km)
    tn, fp, fn, tp = cm.ravel()
    dr = tp/(tp+fn) if tp+fn>0 else 0
    far = fp/(fp+tn) if fp+tn>0 else 0
    print(f"  DR: {dr:.4f}, FAR: {far:.4f}\n  CM:\n{cm}\n")

    y_train_km = np.array([1 if c in attack_clusters else 0 for c in train_labels])
    fp_idx = np.where((y_train_km == 1) & (y_train == 0))[0]
    fn_idx = np.where((y_train_km == 0) & (y_train == 1))[0]

    X_fp = X_train[fp_idx]
    X_fn = X_train[fn_idx]
    X_norm = X_train[y_train == 0]
    X_adv  = X_train[y_train == 1]

    if len(X_fp) and len(X_adv):
        attack_samples_for_fp = X_adv[np.random.choice(len(X_adv), size=len(X_fp),replace=True)]
        Xp = np.concatenate([X_fp, attack_samples_for_fp])
        yp = np.concatenate([np.zeros(len(X_fp)), np.ones(len(X_fp))])
        yp = yp.astype(np.int64)
        y_counts = np.bincount(yp)
        print(y_counts)
        min_groups = np.min(y_counts)
        if min_groups >= 2:
          opt_rfp = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfp.fit(Xp, yp)
          rfp = opt_rfp.best_estimator_
        else:
          rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    else:
        rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)

    if len(X_fn) and len(X_norm):
        normal_samples_for_fn = X_norm[np.random.choice(len(X_norm), size=len(X_fn),replace=True)]
        Xn = np.concatenate([X_fn, normal_samples_for_fn])
        yn = np.concatenate([np.zeros(len(X_fn)), np.ones(len(X_fn))])
        yn = yn.astype(np.int64)
        y_counts = np.bincount(yn)
        print(y_counts)
        min_groups = np.min(y_counts)

        if min_groups >= 2:
          opt_rfn = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfn.fit(Xn, yn)
          rfn = opt_rfn.best_estimator_
        else:
          rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback
    else:
        rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback

    probs = np.array([cluster_prob.get(c,0.0) for c in test_labels])
    best_thr, best_rec = 0.5, 0.0
    for thr in np.linspace(0.5, 0.99, 50):
        y_tmp = y_km.copy()
        for i, p in enumerate(probs):
            if p < thr:
                if y_tmp[i] == 0:
                    y_tmp[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
                else:
                    y_tmp[i] = rfp.predict(X_test[i].reshape(1,-1))[0]
        rec = recall_score(y_test, y_tmp)
        if rec > best_rec:
            best_rec, best_thr = rec, thr

    y_final = y_km.copy()
    for i, p in enumerate(probs):
        if p < best_thr:
            if y_final[i] == 0:
                y_final[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
            else:
                y_final[i] = rfp.predict(X_test[i].reshape(1,-1))[0]


    print("MTH-IDS Performance:")
    print(classification_report(y_test, y_final))
    cm2 = confusion_matrix(y_test, y_final)

    tn, fp, fn, tp = cm2.ravel()
    dr2 = tp/(tp+fn) if tp+fn>0 else 0
    far2 = fp/(fp+tn) if fp+tn>0 else 0
    acc2 = accuracy_score(y_test, y_final)

    print(f"  Acc: {acc2:.4f}, DR: {dr2:.4f}, FAR: {far2:.4f}\n  CM:\n{cm2}")

    return acc2, dr2, far2, cm2

In [261]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={1:25794})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [262]:
one_acc, one_dr, one_far, one_f1 = Anomaly_IDS(X_train, X_test, y_train, y_test, res_gp.x[0])

CL-k-means Performance:
              precision    recall  f1-score   support

           0       0.76      0.73      0.74       568
           1       0.74      0.77      0.75       568

    accuracy                           0.75      1136
   macro avg       0.75      0.75      0.75      1136
weighted avg       0.75      0.75      0.75      1136

  DR: 0.7694, FAR: 0.2746
  CM:
[[412 156]
 [131 437]]

[2925 2925]
[988 988]
MTH-IDS Performance:
              precision    recall  f1-score   support

           0       0.76      0.73      0.74       568
           1       0.74      0.77      0.75       568

    accuracy                           0.75      1136
   macro avg       0.75      0.75      0.75      1136
weighted avg       0.75      0.75      0.75      1136

  Acc: 0.7474, DR: 0.7694, FAR: 0.2746
  CM:
[[412 156]
 [131 437]]
